In [1]:
import pandas as pd
from pandasql import sqldf
import numpy as np

In [2]:
# В таблице user_actions сохраняются действия покупателей в интернет магазине, структура таблицы

# user_id    - id пользователя
# product_id - id товара
# action     - действие, просмотр товара, сохранение в корзину, покупка
# time       - время совершения события
# date       - дата события

# Для каждого дня рассчитайте, какой процент просмотров товаров завершился покупкой

In [3]:
all_user_ids = np.arange(1, 1001)
all_product_ids = np.arange(1, 101)

In [4]:
n = 10000

In [5]:
user_ids    = np.random.choice(all_user_ids, n)
product_ids = np.random.choice(all_product_ids, n)

In [6]:
start_date = pd.to_datetime('2022-01-01')

In [7]:
times = pd.date_range(start_date, periods=n, freq='1min')

In [8]:
user_actions = pd.DataFrame({'user_id': user_ids, 
                             'product_id': product_ids, 
                             'time': times})

In [9]:
user_actions['action'] = 'view'

In [10]:
user_actions.head()

,user_id,product_id,time,action
0,912,4,2022-01-01 00:00:00,view
1,994,91,2022-01-01 00:01:00,view
2,612,82,2022-01-01 00:02:00,view
3,830,58,2022-01-01 00:03:00,view
4,985,46,2022-01-01 00:04:00,view


In [11]:
def generate_funel_actions(user_id, product_id, time):
    to_cart = 0.2
    to_purchase = 0.4
    
    df = pd.DataFrame()
    
    if np.random.binomial(1, to_cart, 1)[0]:
        df = pd.DataFrame({
                           'user_id'   : user_id, 
                           'product_id': product_id, 
                           'time'      : time + pd.Timedelta(5, unit='s'), 
                           'action'    : 'add to cart'}, index=[0])
        
        if np.random.binomial(1, to_purchase, 1)[0]:
            df_purchase = pd.DataFrame({
                           'user_id'   : user_id, 
                           'product_id': product_id, 
                           'time'      : time + pd.Timedelta(10, unit='s'), 
                           'action'    : 'purchase'}, index=[0])
    
            df = df.append(df_purchase)
    return df

In [12]:
to_cart_df = pd.DataFrame()

In [13]:
for index, row in user_actions.iterrows():
    user_df = generate_funel_actions(row['user_id'], row['product_id'], row['time'])
    # to_cart_df = to_cart_df.append(user_df)
    to_cart_df = pd.concat([to_cart_df,user_df])

C:\Users\al\AppData\Local\Temp\ipykernel_22180\2197806347.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_purchase)
C:\Users\al\AppData\Local\Temp\ipykernel_22180\2197806347.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_purchase)
C:\Users\al\AppData\Local\Temp\ipykernel_22180\2197806347.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_purchase)
C:\Users\al\AppData\Local\Temp\ipykernel_22180\2197806347.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_purchase)
C:\Users\al\AppData\Local\Temp\ipykernel_22180\2197806347.py:21: FutureWarning: The fram

In [14]:
user_actions = user_actions.append(to_cart_df)
user_actions = user_actions.sort_values('time')

C:\Users\al\AppData\Local\Temp\ipykernel_22180\3036938177.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  user_actions = user_actions.append(to_cart_df)


In [15]:
user_actions['date'] = user_actions.time.dt.date

In [16]:
user_actions.head()

,user_id,product_id,time,action,date
0,912,4,2022-01-01 00:00:00,view,2022-01-01
0,912,4,2022-01-01 00:00:05,add to cart,2022-01-01
0,912,4,2022-01-01 00:00:10,purchase,2022-01-01
1,994,91,2022-01-01 00:01:00,view,2022-01-01
2,612,82,2022-01-01 00:02:00,view,2022-01-01


In [17]:
user_actions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12732 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     12732 non-null  int64         
 1   product_id  12732 non-null  int64         
 2   time        12732 non-null  datetime64[ns]
 3   action      12732 non-null  object        
 4   date        12732 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 596.8+ KB


In [20]:
q = """SELECT date, 
              views, 
              carts, 
              purchases, 
              100 * purchases / views as purchase_percantage 
       FROM (
           SELECT date, 
            count(case when action = 'view'        then 1 else NULL end) as views, 
            count(case when action = 'add to cart' then 1 else NULL end) as carts, 
            count(case when action = 'purchase'    then 1 else NULL end) as purchases 
           FROM user_actions 
           GROUP BY date) t1;"""


In [23]:
q1 = '''SELECT date FROM user_actions;'''
sqldf(q1)

ObjectNotExecutableError: Not an executable object: 'SELECT date FROM user_actions;'

In [24]:
user_actions.columns

Index(['user_id', 'product_id', 'time', 'action', 'date'], dtype='object')

In [25]:
dfu = user_actions[['date','action']].pivot_table(
    index=['date'],
    columns=['action'],
    aggfunc=len
).reset_index()

In [26]:
dfu['purchase_share'] = 100*dfu['add to cart']/dfu.view

In [27]:
dfu

action,date,add to cart,purchase,view,purchase_share
0,2022-01-01,282,121,1440,19.583333
1,2022-01-02,292,110,1440,20.277778
2,2022-01-03,281,118,1440,19.513889
3,2022-01-04,299,126,1440,20.763889
4,2022-01-05,284,108,1440,19.722222
5,2022-01-06,265,99,1440,18.402778
6,2022-01-07,252,95,1360,18.529412
